In [493]:
%pip install anytree

Note: you may need to restart the kernel to use updated packages.


In [494]:
# Data Structures

import anytree

class Node(anytree.NodeMixin):
    def __init__(self, node_name:str, node_type:str, node_path:str, node_size:int=0, length:int=1, width:int=1, parent=None, children=None):
    
        self.node_name = node_name
        self.node_type = node_type
        self.node_path = node_path
        self.node_size = node_size
        self.length = length
        self.width = width
        self.parent = parent
        
        # if children:
        #     self.children = children
            
    # https://blog.teclado.com/property-decorator-in-python/
    @property
    def node_size(self):
        if self.node_type == "file":
            return self._node_size
        else:
            return sum(child.node_size for child in self.children)
        
    @node_size.setter
    def node_size(self, new_node_size):
        self._node_size = new_node_size
               
    def __repr__(self):
        return f"{self.node_type.upper()}: {self.node_name} ({self.node_size} bytes)"
   

In [495]:
# Read our file input.

from pprint import pprint

with open("input.txt") as command_set:
    # Move pointer to start of file.
    command_set.seek(0)
    
    # Read all lines to a list without new line spacing.
    commands = command_set.read().splitlines()

In [496]:
def change_directory(command:str):
    """This method is used to virtually 'change directory', by appending and popping paths from a global list called 'DIRECTORY_POINTER_LIST.'

    Args:
        command (str): The directory to change to, or '..' to go back.
    """
    
    # If the command is to go back, we pop the last entry in our list.
    if command == "..":
        DIRECTORY_POINTER_LIST.pop()

    else:
        # Otherwise, we add it to our list.
        DIRECTORY_POINTER_LIST.append(command)

In [497]:
# Create a directory pointer list.
DIRECTORY_POINTER_LIST = []

# Create a root node and set it as index 0 to our nodes_list.
nodes_list = [Node("/", "dir", "/")]

for line in commands:
    if line.startswith('$ ls'):
        continue

    elif line.startswith('$ cd'):  # Deal with changing directories.
        # Change directory to the last substring in our string.
        change_directory(line.split()[-1])

    elif line.startswith('dir'):  # Deal with directory ls'es.
        dir_name = line.split()[-1]

        # Create a full path from our pointer list.
        base_path = "/".join(DIRECTORY_POINTER_LIST).replace("//", "/")
        dir_path = f"{base_path}/{dir_name}".replace("//", "/").replace("//", "/")

        # Pick parent with the same basepath as this current new node.
        parent = next((node for node in nodes_list if node.node_path == base_path), None)
        
        # Append it to our nodes_list.   
        nodes_list.append(Node(dir_name, "dir", dir_path, parent=parent))

    else:  # Deal with file ls'es.
        file_size, file_name = line.split()

        # Create a full path from our pointer list.
        base_path = "/".join(DIRECTORY_POINTER_LIST).replace("//", "/")
        file_path = f"{base_path}/{file_name}".replace("//", "/")

        # Pick parent with the same basepath as this current new node.
        parent = next((node for node in nodes_list if node.node_path == base_path), None)

        # Append it to our nodes_list.
        nodes_list.append(Node(file_name, "file", file_path, int(file_size), parent=parent))
        
print(anytree.RenderTree(nodes_list[0]))


DIR: / (46876531 bytes)
├── DIR: ccjp (1561333 bytes)
│   ├── FILE: dlz (159990 bytes)
│   ├── DIR: mbtsvblj (1236267 bytes)
│   │   ├── FILE: frqsf.nsv (34806 bytes)
│   │   ├── DIR: ppq (266252 bytes)
│   │   │   └── FILE: dhzp (266252 bytes)
│   │   ├── DIR: ptht (653838 bytes)
│   │   │   ├── DIR: jbnj (395311 bytes)
│   │   │   │   ├── DIR: clscr (249531 bytes)
│   │   │   │   │   └── FILE: dhzp (249531 bytes)
│   │   │   │   └── FILE: zwtwf.zfz (145780 bytes)
│   │   │   └── DIR: zcbnwhzd (258527 bytes)
│   │   │       └── DIR: mbtsvblj (258527 bytes)
│   │   │           └── FILE: pjhvzjt.brz (258527 bytes)
│   │   └── DIR: rgmvdwt (281371 bytes)
│   │       ├── FILE: hdb (166021 bytes)
│   │       └── FILE: ljpzpdbf (115350 bytes)
│   └── FILE: nppbjl.qhg (165076 bytes)
├── FILE: hglnvs.bsh (328708 bytes)
├── DIR: hpsnpc (318283 bytes)
│   ├── FILE: bfdfbrh.vff (6334 bytes)
│   └── FILE: qphlw.whm (311949 bytes)
├── DIR: pcb (6258197 bytes)
│   ├── DIR: ccjp (3698153 bytes)
│   

In [498]:
answer_1_nodes = anytree.search.findall(nodes_list[0], filter_=lambda node: node.node_size <= 100000 and node.node_type == "dir")

answer_1_value = sum(node.node_size for node in answer_1_nodes)

print(answer_1_value)

1513699


In [499]:
total_disk_space = 70000000
used_disk_space = nodes_list[0].node_size
free_disk_space = total_disk_space - used_disk_space

print(free_disk_space)

required_update_space = 30000000
min_space_to_delete = required_update_space - free_disk_space

print(required_update_space)
print(min_space_to_delete)

def get_size(node:Node):
    return node.node_size

answer_2_nodes = anytree.search.findall(nodes_list[0], filter_=lambda node: node.node_size >= min_space_to_delete and node.node_size <= required_update_space and node.node_type == "dir")

# Answer is the lowest of these nodes.
print(answer_2_nodes)

# 7991939


23123469
30000000
6876531
(DIR: pntzm (29757629 bytes), DIR: mbtsvblj (27924493 bytes), DIR: csqcnmtc (9405481 bytes), DIR: mbtsvblj (7991939 bytes), DIR: zcrrtlh (14682970 bytes), DIR: dhsmmlt (13246500 bytes), DIR: wffbp (12324512 bytes), DIR: mbtsvblj (10264814 bytes), DIR: mbtsvblj (8113815 bytes), DIR: thfgwwsp (8389675 bytes))
